In [2]:
from pysr import PySRRegressor

Detected Jupyter notebook. Loading juliacall extension. Set `PYSR_AUTOLOAD_EXTENSIONS=no` to disable.


In [1]:
import pmlb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_pinball_loss
import uncertainty_toolbox

In [4]:
import torch

In [2]:
from pmlb import fetch_data, regression_dataset_names

In [6]:
import numpy as np

def get_feature_type(data_column):
  """
  This function determines the feature type (categorical or binary) for a NumPy array column.

  **Note:** This function assumes the data doesn't contain missing values.
          If your data might have missing values, you'll need to handle them
          before using this function (e.g., impute missing values or remove rows).

  Args:
      data_column: A NumPy array representing the data column.

  Returns:
      A string indicating the feature type: "categorical" or "binary".
  """

  # Check for distinct values and data type
  unique_values = np.unique(data_column)
  num_unique_values = len(unique_values)

  # Categorical data has a limited number of distinct values (adjust threshold as needed)
  if num_unique_values <= 10:  # Adjust threshold based on your data and analysis goals
    return "categorical"

  # Binary data has only two distinct values
  if num_unique_values == 2:
    return "binary"

  # If there are more than 10 distinct values and not binary, assume numerical
  # (This might need further refinement depending on your domain knowledge)
  return "numerical"  # Consider a different label for non-categoric

In [7]:
def get_categorical_features(X):
  """
  This function identifies the indices of categorical features in a NumPy array representing a dataset.

  **Note:** This function assumes the data doesn't contain missing values.
          If your data might have missing values, you'll need to handle them
          before using this function (e.g., impute missing values or remove rows).

  Args:
      X: A 2D NumPy array representing the dataset (n_samples x n_features).

  Returns:
      A list of integers representing the indices of categorical features in X.
  """

  categorical_features = []
  for i, col in enumerate(X.T):  # Enumerate to get column index (i)
    unique_values = np.unique(col)
    num_unique_values = len(unique_values)
    data_type = col.dtype

    # Categorical data has a limited number of distinct values (adjust threshold as needed)
    if num_unique_values <= 10:  # Adjust threshold based on your data and analysis goals
      categorical_features.append(i)

  return categorical_features

In [8]:
from sklearn.preprocessing import OneHotEncoder

def create_dummy_variables(X, categorical_features):
    """
    This function creates dummy variables for categorical features in a dataset.

    Args:
        X: A 2D NumPy array representing the dataset (n_samples x n_features).
        categorical_features: A list of integers representing the indices of categorical features in X.

    Returns:
        A 2D NumPy array representing the dataset with dummy variables for categorical features.
    """
    if categorical_features == []:
        return X
    else:
        # Select categorical features from the data
        X_categorical = X[:, categorical_features]

        # Create one-hot encoder
        encoder = OneHotEncoder(sparse=False)  # Set sparse=False for dense output

        # Fit the encoder on the categorical features
        encoder.fit(X_categorical)

        # Transform the categorical features into dummy variables
        X_dummy_categorical = encoder.transform(X_categorical)

        # Get the original non-categorical features (assuming they are numerical)
        X_numerical = np.delete(X, categorical_features, axis=1)  # Delete categorical feature columns

        # Combine the dummy variables and numerical features
        X_with_dummies = np.concatenate([X_numerical, X_dummy_categorical], axis=1)

        return X_with_dummies


In [10]:
regressiondatasets = regression_dataset_names

**SQR**

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sb
from pysr import PySRRegressor
from pmlb import fetch_data, regression_dataset_names

def pinball_loss(y_true, y_pred, tau=0.9):
    residuals = y_true - y_pred
    # Calculate loss for each pair of true and predicted values
    loss = np.where(residuals >= 0, tau * residuals, (1 - tau) * -residuals)
    # Compute the mean loss across all predictions
    return np.mean(loss)

test_losses = []
test_coverage = []
test_expressions = []

for regression_dataset in regressiondatasets:
    print(regression_dataset)
    X1, y = fetch_data(regression_dataset, return_X_y=True)
    print(X1)
    X = create_dummy_variables(X1, get_categorical_features(X1))
    print(X)
    kf = KFold(n_splits=5, shuffle=True)
    losses_fold = np.zeros(5)
    coverage_fold = np.zeros(5)
    expressions = ["", "", "", "", ""]
    for i, (train_index, test_index) in enumerate(kf.split(X)):
        train_X, test_X = X[train_index], X[test_index]
        print(train_X)
        train_y, test_y = y[train_index], y[test_index]

        modelq = PySRRegressor(
        niterations=100,  # < Increase me for better results
        binary_operators=["+", "*", "/", "-"],
        unary_operators=["exp", "sin", "cos", "log", "square"],
        complexity_of_operators={"+": 1, "-": 1, "*": 1,  "/": 2, "exp": 4, "sin": 3, "cos": 3, "log": 3, "square": 2},
        # ^ Define operator for SymPy as well
        elementwise_loss="pinball_loss(y_true, y_pred) = max.(0.1 * (y_true - y_pred), (0.1 - 1) * (y_true - y_pred))"

, 
        # ^ Custom loss function (julia syntax)
        temp_equation_file=True
)

        modelq.fit(train_X, train_y)

        y_pred_symbolic = modelq.predict(test_X)
        losses_fold[i] = pinball_loss(test_y, y_pred_symbolic)
        coverage_fold[i] = np.mean(y_pred_symbolic >= test_y)
        expressions[i] = modelq.sympy()

    test_losses.append(losses_fold)
    test_coverage.append(coverage_fold)
    test_expressions.append(expressions)

# Display the results

print(test_losses)
print(test_coverage)
print(test_expressions)


1027_ESL
[[6. 5. 6. 6.]
 [5. 4. 5. 5.]
 [5. 3. 4. 5.]
 ...
 [4. 4. 6. 8.]
 [8. 6. 6. 7.]
 [7. 6. 5. 5.]]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Compiling Julia backend...


c:\Users\k100814\Anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\k100814\Anaconda\Lib\site-packages\pysr\sr.py:1297: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\k100814\Anaconda\Lib\site-packages\pysr\sr.py:1874: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, or, alternatively, do a dimensionality reduction beforehand. For example, `X = PCA(n_components=6).fit_transform(X)`, using scikit-learn's `PCA` class, will reduce the number of features to 6 in an interpretable way, as each resultant feature will be a linea


Expressions evaluated per second: 2.100e+05
Head worker occupation: 31.1%. This is high, and will prevent efficient resource usage. Increase `ncyclesperiteration` to reduce load on head worker.
Progress: 430 / 1500 total iterations (28.667%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           2.357e-01  1.594e+01  y = 7.0002
3           2.051e-01  6.934e-02  y = (7.0002 - x₃₀)
5           1.683e-01  9.888e-02  y = ((x₃₂ - -6.0025) + x₈)
7           1.520e-01  5.109e-02  y = ((x₃₂ - (-6.0025 - x₂₅)) + x₈)
9           1.368e-01  5.277e-02  y = (((x₃₂ - x₂₂) + x₂₅) + (x₈ - -6.0084))
11          1.224e-01  5.555e-02  y = (((x₃₂ - (x₄ + x₂₂)) + (x₈ - -6.0084)) + x₂₅)
13          1.163e-01  2.552e-02  y = (((x₃₂ - ((x₄ + x₂₂) - x₁₈)) + (x₈ - -6.0084)) + x₂₅)
15          1.107e-01  2.469e-02  y = ((((((x₃₂ + (x₈ - -6.0187)) + x₁₈) - x₂₉) - x₃₀) + x₇) - x...
                    

c:\Users\k100814\Anaconda\Lib\site-packages\pysr\sr.py:1297: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\k100814\Anaconda\Lib\site-packages\pysr\sr.py:1874: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, or, alternatively, do a dimensionality reduction beforehand. For example, `X = PCA(n_components=6).fit_transform(X)`, using scikit-learn's `PCA` class, will reduce the number of features to 6 in an interpretable way, as each resultant feature will be a linear combination of the original features. 
  warnings.warn(
[ Info: Started!


[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


c:\Users\k100814\Anaconda\Lib\site-packages\pysr\sr.py:1297: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\k100814\Anaconda\Lib\site-packages\pysr\sr.py:1874: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, or, alternatively, do a dimensionality reduction beforehand. For example, `X = PCA(n_components=6).fit_transform(X)`, using scikit-learn's `PCA` class, will reduce the number of features to 6 in an interpretable way, as each resultant feature will be a linear combination of the original features. 
  warnings.warn(
[ Info: Started!


In [11]:
regression_dataset_namestry = regression_dataset_names

**LIGHTGBM**

In [12]:
from sklearn.model_selection import KFold
import lightgbm as lgb
from pmlb import fetch_data
import optuna
import pandas as pd
from datetime import datetime
import optuna

def pinball_loss(y_true, y_pred, tau=0.9):
    residuals = y_true - y_pred
    # Calculate loss for each pair of true and predicted values
    loss = np.where(residuals >= 0, tau * residuals, (1 - tau) * -residuals)
    # Compute the mean loss across all predictions
    return np.mean(loss)

def objective(trial, train_X, train_y, val_X, val_y):
    params = {
        'objective': 'quantile',
        'alpha': 0.9,
        'num_leaves': trial.suggest_int('num_leaves', 2, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
    if params['min_child_samples'] >= params['num_leaves']:
        raise optuna.exceptions.TrialPruned()

    model = lgb.LGBMRegressor(**params)
    model.fit(train_X, train_y)
    y_pred = model.predict(val_X)
    return pinball_loss(val_y, y_pred, tau=0.9)

results = []

for regression_dataset in regression_dataset_names:  # Make sure this variable is properly initialized
    X1, y = fetch_data(regression_dataset, return_X_y=True)
    print(X1)
    X = create_dummy_variables(X1, get_categorical_features(X1))
    print(X)
    kf = KFold(n_splits=5, shuffle=True)
    dataset_results = []

    for train_index, val_index in kf.split(X):
        train_X, val_X = X[train_index], X[val_index]
        train_y, val_y = y[train_index], y[val_index]

        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective(trial, train_X, train_y, val_X, val_y), n_trials=10)

        best_params = study.best_params
        model = lgb.LGBMRegressor(objective='quantile', alpha=0.9, **best_params)
        model.fit(train_X, train_y)
        y_pred = model.predict(val_X)
        loss = pinball_loss(val_y, y_pred, tau=0.9)
        coverage = np.mean(y_pred >= val_y)
        dataset_results.append({'Loss': loss, 'Coverage': coverage})

    results.append({'Dataset': regression_dataset, 'Results': dataset_results})


c:\Users\k100814\Anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
[I 2024-07-12 10:57:56,777] A new study created in memory with name: no-name-ac26ddfa-a5a0-4921-9414-c1a6d89e8735


[[6. 5. 6. 6.]
 [5. 4. 5. 5.]
 [5. 3. 4. 5.]
 ...
 [4. 4. 6. 8.]
 [8. 6. 6. 7.]
 [7. 6. 5. 5.]]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


[I 2024-07-12 10:57:57,195] Trial 0 finished with value: 0.08441432664063735 and parameters: {'num_leaves': 95, 'learning_rate': 0.08773027378854753, 'max_depth': 5, 'min_child_samples': 18}. Best is trial 0 with value: 0.08441432664063735.
[I 2024-07-12 10:57:57,210] Trial 1 finished with value: 0.19489795918367347 and parameters: {'num_leaves': 75, 'learning_rate': 0.45158840574616704, 'max_depth': 3, 'min_child_samples': 68}. Best is trial 0 with value: 0.08441432664063735.
[I 2024-07-12 10:57:57,211] Trial 2 pruned. 
[I 2024-07-12 10:57:57,213] Trial 3 pruned. 
[I 2024-07-12 10:57:57,214] Trial 4 pruned. 
[I 2024-07-12 10:57:57,216] Trial 5 pruned. 
[I 2024-07-12 10:57:57,237] Trial 6 finished with value: 0.1001839224424603 and parameters: {'num_leaves': 83, 'learning_rate': 0.1203315753690219, 'max_depth': 18, 'min_child_samples': 29}. Best is trial 0 with value: 0.08441432664063735.
[I 2024-07-12 10:57:57,259] Trial 7 finished with value: 0.10546893131510846 and parameters: {'num

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000064 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 42
[LightGBM] [Info] Number of data points in the train set: 390, number of used features: 21
[LightGBM] [Info] Start training from score 7.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

[I 2024-07-12 10:57:57,357] Trial 4 finished with value: 0.11858845664204198 and parameters: {'num_leaves': 49, 'learning_rate': 0.06972949486539867, 'max_depth': 20, 'min_child_samples': 35}. Best is trial 4 with value: 0.11858845664204198.
[I 2024-07-12 10:57:57,360] Trial 5 pruned. 
[I 2024-07-12 10:57:57,383] Trial 6 finished with value: 0.15437901247553998 and parameters: {'num_leaves': 90, 'learning_rate': 0.23826631218859273, 'max_depth': 3, 'min_child_samples': 66}. Best is trial 4 with value: 0.11858845664204198.
[I 2024-07-12 10:57:57,385] Trial 7 pruned. 
[I 2024-07-12 10:57:57,409] Trial 8 finished with value: 0.17127731635337323 and parameters: {'num_leaves': 52, 'learning_rate': 0.3055304213624483, 'max_depth': 12, 'min_child_samples': 44}. Best is trial 4 with value: 0.11858845664204198.
[I 2024-07-12 10:57:57,421] Trial 9 finished with value: 0.15584215532540585 and parameters: {'num_leaves': 61, 'learning_rate': 0.03350654652525427, 'max_depth': 2, 'min_child_samples':

...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029347 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2588
[LightGBM] [Info] Number of data points in the train set: 800000, number of used features: 29
[LightGBM] [Info] Start training from score 3625.940430


[I 2024-07-12 10:59:24,691] Trial 5 finished with value: 153.79428389282066 and parameters: {'num_leaves': 96, 'learning_rate': 0.22657439276150515, 'max_depth': 14, 'min_child_samples': 88}. Best is trial 5 with value: 153.79428389282066.
[I 2024-07-12 10:59:24,694] Trial 6 pruned. 
[I 2024-07-12 10:59:24,721] Trial 7 pruned. 
[I 2024-07-12 10:59:24,724] Trial 8 pruned. 


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.033746 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2588
[LightGBM] [Info] Number of data points in the train set: 800000, number of used features: 29
[LightGBM] [Info] Start training from score 3625.940430


[W 2024-07-12 10:59:27,313] Trial 9 failed with parameters: {'num_leaves': 88, 'learning_rate': 0.12051379970670703, 'max_depth': 9, 'min_child_samples': 45} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\k100814\Anaconda\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\k100814\AppData\Local\Temp\ipykernel_85672\4043318971.py", line 49, in <lambda>
    study.optimize(lambda trial: objective(trial, train_X, train_y, val_X, val_y), n_trials=10)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\k100814\AppData\Local\Temp\ipykernel_85672\4043318971.py", line 30, in objective
    model.fit(train_X, train_y)
  File "c:\Users\k100814\Anaconda\Lib\site-packages\lightgbm\sklearn.py", line 1092, in fit
    super().fit(
  File "c:\Users\k100814\Anaconda\Lib\site-packages\lightgbm\sklearn

KeyboardInterrupt: 

**QDT**

In [13]:
from sklearn.tree import DecisionTreeRegressor
import optuna
from pmlb import fetch_data, regression_dataset_names
from sklearn.model_selection import KFold
import pandas as pd

class QuantileDecisionTreeRegressor:
    def __init__(self, quantile=0.9, min_samples_leaf=5):
        self.quantile = quantile
        self.min_samples_leaf = min_samples_leaf
        self.tree = DecisionTreeRegressor(min_samples_leaf=min_samples_leaf)

    def fit(self, X, y):
        self.tree.fit(X, y)
        self._add_quantile_info(X, y)

    def _add_quantile_info(self, X, y):
        leaf_indices = self.tree.apply(X)
        unique_leaves = np.unique(leaf_indices)
        self.quantile_values = {}
        for leaf in unique_leaves:
            leaf_y = y[leaf_indices == leaf]
            self.quantile_values[leaf] = np.percentile(leaf_y, self.quantile * 100)

    def predict(self, X):
        leaf_indices = self.tree.apply(X)
        predictions = np.array([self.quantile_values[leaf] for leaf in leaf_indices])
        return predictions

def pinball_loss(y_true, y_pred, tau=0.9):
    residuals = y_true - y_pred
    loss = np.where(residuals >= 0, tau * residuals, (1 - tau) * -residuals)
    return np.mean(loss)

def objective(trial, train_X, train_y, val_X, val_y):
    quantile = 0.9
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 100)

    model = QuantileDecisionTreeRegressor(quantile=quantile, min_samples_leaf=min_samples_leaf)
    model.fit(train_X, train_y)
    y_pred = model.predict(val_X)

    return pinball_loss(val_y, y_pred, tau=quantile)

# Example dataset usage
regression_dataset_tryout = regression_dataset_names
results = []

for regression_dataset in regression_dataset_tryout:
    X, y = fetch_data(regression_dataset, return_X_y=True)
    kf = KFold(n_splits=5, shuffle=True)
    dataset_results = []

    for train_index, val_index in kf.split(X):
        train_X, val_X = X[train_index], X[val_index]
        train_y, val_y = y[train_index], y[val_index]

        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective(trial, train_X, train_y, val_X, val_y), n_trials=10)

        best_params = study.best_params
        best_model = QuantileDecisionTreeRegressor(**best_params)
        best_model.fit(train_X, train_y)
        y_pred = best_model.predict(val_X)

        loss = pinball_loss(val_y, y_pred, tau=best_model.quantile)
        coverage = np.mean(y_pred >= val_y)

        dataset_results.append({'Loss': loss, 'Coverage': coverage})

    results.append({'Dataset': regression_dataset, 'Results': dataset_results})


[I 2024-07-12 10:59:43,417] A new study created in memory with name: no-name-ba6d8598-040a-413f-a4d6-527da263cf07
[I 2024-07-12 10:59:43,421] Trial 0 finished with value: 0.14795918367346936 and parameters: {'min_samples_leaf': 58}. Best is trial 0 with value: 0.14795918367346936.
[I 2024-07-12 10:59:43,424] Trial 1 finished with value: 0.2071428571428571 and parameters: {'min_samples_leaf': 99}. Best is trial 0 with value: 0.14795918367346936.
[I 2024-07-12 10:59:43,427] Trial 2 finished with value: 0.1259183673469388 and parameters: {'min_samples_leaf': 27}. Best is trial 2 with value: 0.1259183673469388.
[I 2024-07-12 10:59:43,430] Trial 3 finished with value: 0.14489795918367343 and parameters: {'min_samples_leaf': 74}. Best is trial 2 with value: 0.1259183673469388.
[I 2024-07-12 10:59:43,434] Trial 4 finished with value: 0.11061224489795918 and parameters: {'min_samples_leaf': 36}. Best is trial 4 with value: 0.11061224489795918.
[I 2024-07-12 10:59:43,438] Trial 5 finished with 

**LINEAR**

In [4]:
regressiondatasettryout = regression_dataset_names

In [9]:
import pandas as pd
from pmlb import fetch_data
import optuna
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from statsmodels.regression.quantile_regression import QuantReg

def create_dummy_variables(X, categorical_features):
    if not categorical_features:
        return sm.add_constant(X, has_constant='add')
    encoder = OneHotEncoder(sparse=False)
    X_categorical = encoder.fit_transform(X[:, categorical_features])
    X_numerical = np.delete(X, categorical_features, axis=1)
    X_with_dummies = np.concatenate([X_numerical, X_categorical], axis=1)
    return sm.add_constant(X_with_dummies, has_constant='add')

def pinball_loss(y_true, y_pred, tau=0.5):
    residuals = y_true - y_pred
    return np.mean(np.where(residuals >= 0, tau * residuals, (1 - tau) * -residuals))

def objective(trial, train_X, train_y, val_X, val_y, tau=0.5):
    max_iter = trial.suggest_int('max_iter', 1000, 5000)
    model = QuantReg(train_y, train_X)
    results = model.fit(q=tau, max_iter=max_iter)
    y_pred = results.predict(val_X)
    return pinball_loss(val_y, y_pred, tau)

resultslinear = []

for regression_dataset in regressiondatasettryout:
    print(regression_dataset)
    X1, y = fetch_data(regression_dataset, return_X_y=True)

    # Apply subsampling for large datasets
    if X1.shape[0] > 500000:
        idx = np.random.choice(X1.shape[0], 100000, replace=False)
        X1 = X1[idx]
        y = y[idx]

    categorical_features = get_categorical_features(X1)
    X = create_dummy_variables(X1, categorical_features)
    kf = KFold(n_splits=3 if X1.shape[0] > 1000000 else 5, shuffle=True)
    dataset_results = []

    for train_index, val_index in kf.split(X):
        train_X, val_X = X[train_index], X[val_index]
        train_y, val_y = y[train_index], y[val_index]

        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective(trial, train_X, train_y, val_X, val_y, tau=0.5), n_trials=5 if X1.shape[0] > 500000 else 10)

        best_params = study.best_trial.params
        final_model = QuantReg(train_y, train_X).fit(q=0.5, max_iter=best_params['max_iter'])
        y_pred = final_model.predict(val_X)
        loss = pinball_loss(val_y, y_pred, tau=0.5)
        coverage = np.mean(y_pred >= val_y)
        dataset_results.append({'Loss': loss, 'Coverage': coverage})

    resultslinear.append({'Dataset': regression_dataset, 'Results': dataset_results})


1027_ESL


c:\Users\k100814\Anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
[I 2024-07-12 11:05:27,020] A new study created in memory with name: no-name-979b88ba-b2cd-4c74-ae14-68eede821d8c
[I 2024-07-12 11:05:27,575] Trial 0 finished with value: 0.19423839004682994 and parameters: {'max_iter': 4308}. Best is trial 0 with value: 0.19423839004682994.
[I 2024-07-12 11:05:27,691] Trial 1 finished with value: 0.19423839004682994 and parameters: {'max_iter': 1021}. Best is trial 0 with value: 0.19423839004682994.
[I 2024-07-12 11:05:27,811] Trial 2 finished with value: 0.19423839004682994 and parameters: {'max_iter': 4727}. Best is trial 0 with value: 0.19423839004682994.
[I 2024-07-12 11:05:27,943] Trial 3 finished with value: 0.19423839004682994 and parameters: {'max_iter': 3813}. Best is trial

1028_SWD


c:\Users\k100814\Anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
[I 2024-07-12 11:05:37,004] A new study created in memory with name: no-name-a8d0ab40-b7ef-41d6-9e92-2714ef6b5e64
[I 2024-07-12 11:05:37,105] Trial 0 finished with value: 0.23443284726829178 and parameters: {'max_iter': 2924}. Best is trial 0 with value: 0.23443284726829178.
[I 2024-07-12 11:05:37,187] Trial 1 finished with value: 0.23443284726829178 and parameters: {'max_iter': 2979}. Best is trial 0 with value: 0.23443284726829178.
[I 2024-07-12 11:05:37,273] Trial 2 finished with value: 0.23443284726829178 and parameters: {'max_iter': 3818}. Best is trial 0 with value: 0.23443284726829178.
[I 2024-07-12 11:05:37,358] Trial 3 finished with value: 0.23443284726829178 and parameters: {'max_iter': 4169}. Best is trial

KeyboardInterrupt: 